# Loading the Libraries

In [ ]:
import osmnx as ox
import csv as csv 
import numpy as np
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import json

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
reg = LogisticRegression()


from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier()

from sklearn import tree
dt= tree.DecisionTreeClassifier()

from sklearn import svm
svmc = svm.SVC()

from pysal.lib import weights
import esda
from splot.esda import moran_scatterplot, lisa_cluster, plot_local_autocorrelation
import contextily as ctx

# Reading the shapefile

In [ ]:
uk = gp.read_file('/Users/ayrus/Desktop/Dissertation/uk/msoa.shp')

In [ ]:
# Removing unwanted columns

uk.drop('OBJECTID', inplace=True, axis=1)
uk.drop('MSOA21NM', inplace=True, axis=1)
uk.drop('GlobalID', inplace=True, axis=1)
uk.drop('SHAPE_Leng', inplace=True, axis=1)

# Reading the census Data

In [ ]:
msoa = pd.read_csv('/Users/ayrus/Desktop/Dissertation/essex/msoa.csv')

# Integrating shapefile and census data

In [ ]:
# from UK shapefile at MSOA level, we seperated the Essex by merging the Essex MSOA census data

essex = uk.merge(msoa, left_on="MSOA21CD",right_on="MSOA Code")

In [ ]:
# Removing unwanted columns

essex.drop('MSOA21CD', inplace=True, axis=1)
essex.drop('MSOA Name', inplace=True, axis=1)

In [ ]:
# saving the shapefile of essex with integrated spatial and census data

# essex.to_file('/Users/ayrus/Desktop/Dissertation/essex/essex.shp')

# Retrieving the features from OpenStreetMap 

In [ ]:
# changing the Coordinate Reference System (CRS) from EPSG:27700 to EPSG:4326

essex=essex.to_crs({'init': 'epsg:4326'})

### 1. retrieving the "Road Instersections" of Drive type as "nodes"

In [ ]:
nod=[]
for geo in essex['geometry']:
    i = ox.graph_from_polygon(geo, network_type="drive")
    n, e = ox.graph_to_gdfs(i)
    nod.append(len(n))
essex['nodes']=nod

### 2. retrieving the "Residential Buildings" as "res_blds"

In [ ]:
r_bld = {'building':['apartments','bungalow','cabin','detached','house','residential','semidetached_house','terrace']}
rb1=[]
for geo in essex['geometry'][:100]:
    i = ox.geometries_from_polygon(geo, tags=r_bld)
    rb1.append(len(i))

In [ ]:
r_bld = {'building':['apartments','bungalow','cabin','detached','house','residential','semidetached_house','terrace']}
rb2=[]
for geo in essex['geometry'][100:150]:
    i = ox.geometries_from_polygon(geo, tags=r_bld)
    rb2.append(len(i))

In [ ]:
r_bld = {'building':['apartments','bungalow','cabin','detached','house','residential','semidetached_house','terrace']}
rb3=[]
for geo in essex['geometry'][150:175]:
    i = ox.geometries_from_polygon(geo, tags=r_bld)
    rb3.append(len(i))

In [ ]:
essex['res_blds']=rb1+rb2+rb3

### 3. retrieving the "Commercial Buildings" as "com_blds"

In [ ]:
c_bld = {'building':['commercial','industrial','kiosk','office','retail','supermarket','warehouse']}
cb1=[]
for geo in essex['geometry'][:100]:
    i = ox.geometries_from_polygon(geo, tags=c_bld)
    cb1.append(len(i))

In [ ]:
c_bld = {'building':['commercial','industrial','kiosk','office','retail','supermarket','warehouse']}
cb2=[]
for geo in essex['geometry'][100:175]:
    i = ox.geometries_from_polygon(geo, tags=c_bld)
    cb2.append(len(i))

In [ ]:
essex['com_blds']=cb1+cb2

### 4. retrieving the "Shops" as "shop"

In [ ]:
shop = {'shop':True}
sp1=[]
for geo in essex['geometry'][:100]:
    i = ox.geometries_from_polygon(geo, tags=shop)
    sp1.append(len(i))

In [ ]:
shop = {'shop':True}
sp2=[]
for geo in essex['geometry'][100:175]:
    i = ox.geometries_from_polygon(geo, tags=shop)
    sp2.append(len(i))

In [ ]:
essex['shop']=sp1+sp2

### 5. retrieving the "Tourism Places" as "tour"

In [ ]:
tour = {'tourism':True}
tr=[]
for geo in essex['geometry']:
    i = ox.geometries_from_polygon(geo, tags=tour)
    tr.append(len(i))
essex['tour']=tr

### 6. retrieving the "Places of Worship" as "pow"

In [ ]:
plow = {'amenity':'place_of_worship'}
pw=[]
for geo in essex['geometry']:
    i = ox.geometries_from_polygon(geo, tags=plow)
    pw.append(len(i))
essex['pow']=pw

### 7. retrieving the "Educational Places" as "poi1"

In [ ]:
amen1 = {'amenity':['school','college','university','hospital']}
poi1=[]
for geo in essex['geometry']:
    i = ox.geometries_from_polygon(geo, tags=amen1)
    poi1.append(len(i))
essex['poi1']=poi1

### 8. retrieving the "Places of Sustenance and Entertainment" as "poi2"

In [ ]:
amen2 = {'amenity':['restaurant','cafe','fast_food','food_court','pub','bar','cinema','theatre']}
poi2=[]
for geo in essex['geometry']:
    i = ox.geometries_from_polygon(geo, tags=amen2)
    poi2.append(len(i))
essex['poi2']=poi2

### 9. retrieving the "Green Areas" as "poi3"

In [ ]:
amen3 = {'leisure':['park','garden']}
poi3=[]
for geo in essex['geometry']:
    i = ox.geometries_from_polygon(geo, tags=amen3)
    poi3.append(len(i))
essex['poi3']=poi3

### 10. retrieving the "Fitness and Sports Areas" as "poi4"

In [ ]:
amen4 = {'leisure':['playground','stadium','fitness_centre','fitness_station']}
poi4=[]
for geo in essex['geometry']:
    i = ox.geometries_from_polygon(geo, tags=amen4)
    poi4.append(len(i))
essex['poi4']=poi4

# Preparing the Data for Analysis

In [ ]:
# converting the Area of MSOA polygons from metre square to km square

essex['SHAPE_Area']= round((essex['SHAPE_Area']/1000000),3)

In [ ]:
# Removing unwanted columns

essex.drop('Males', inplace=True, axis=1)
essex.drop('Females', inplace=True, axis=1)

In [ ]:
# converting the numbers into densities by dividing them with their respective areas of each MSOA polygon

for col in essex.columns[4:]:
    essex[col]=essex[col]/essex['SHAPE_Area']

In [ ]:
# assigning the Predictors and Target variables to a dataframe "df"

df=essex[['All Ages','0 - 19','20 - 39','40 - 59','60 +','nodes','com_blds','res_blds','shop','tour','pow','poi1','poi2','poi3','poi4']]

In [ ]:
# assigning the Target variables to a dataframe "Y"

Y=essex[['All Ages','0 - 19','20 - 39','40 - 59','60 +']]

In [ ]:
# assigning the Predictors  to a dataframe "X"

X = essex[['nodes','com_blds','res_blds','shop','tour','pow','poi1','poi2','poi3','poi4']]

# Correlation Plots

In [ ]:
df.corr()

In [ ]:
df.corr(method="kendall")

In [ ]:
sns.heatmap(df.corr())

In [ ]:
sns.heatmap(df.corr(method="kendall"))

# Choropleth Plots

In [ ]:
# saving the shape file as geojson file

# essex.to_file('/Users/ayrus/Desktop/Dissertation/json/essex.geojson', driver='GeoJSON')

In [ ]:
# reading the geojson file

j=open('/Users/ayrus/Desktop/Dissertation/json/essex.geojson')
dff= json.load(j)

In [ ]:
q_y0_1 = essex["All Ages"].quantile(0.1)
q_y0_2 = essex["All Ages"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="All Ages",
                           range_color=(q_y0_1,q_y0_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
q_y1_1 = essex["0 - 19"].quantile(0.1)
q_y1_2 = essex["0 - 19"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="0 - 19",
                           range_color=(q_y1_1,q_y1_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
q_y2_1 = essex["20 - 39"].quantile(0.1)
q_y2_2 = essex["20 - 39"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="20 - 39",
                           range_color=(q_y2_1,q_y2_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
q_y3_1 = essex["40 - 59"].quantile(0.1)
q_y3_2 = essex["40 - 59"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="40 - 59",
                           range_color=(q_y3_1,q_y3_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
q_y4_1 = essex["60 +"].quantile(0.1)
q_y4_2 = essex["60 +"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="60 +",
                           range_color=(q_y4_1,q_y4_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
q_x1_1 = essex["nodes"].quantile(0.1)
q_x1_2 = essex["nodes"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="nodes",
                           range_color=(q_x1_1,q_x1_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
q_x2_1 = essex["res_blds"].quantile(0.1)
q_x2_2 = essex["res_blds"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="res_blds",
                           range_color=(q_x2_1,q_x2_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
q_x4_1 = essex["shop"].quantile(0.1)
q_x4_2 = essex["shop"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="shop",
                           range_color=(q_x4_1,q_x4_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
q_x7_1 = essex["poi1"].quantile(0.1)
q_x7_2 = essex["poi1"].quantile(0.9)

In [ ]:
fig = px.choropleth_mapbox(essex,
                           locations="MSOA Code",
                           geojson=dff,featureidkey="properties.MSOA Code",
                            color="poi1",
                           range_color=(q_x7_1,q_x7_2),
                           center={"lat": 51.8, "lon": 0.6},
                           labels={'shop':'shops'},
                           opacity=0.95,
                           mapbox_style="carto-positron", zoom=8.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Multiple Linear Regression (MLR) and AIC

In [ ]:
# feature selection method

sfs1 = sfs(lr,
           k_features=10,
           forward=True,
           floating=False,
           verbose=2,
           scoring=None
           ,cv=5
          )

### Splitting the data into Train and Test

In [ ]:
X_train, X_test, y0_train, y0_test = train_test_split( X, Y['All Ages'], test_size=0.33, random_state=1234)

In [ ]:
X_train, X_test, y1_train, y1_test = train_test_split( X, Y['0 - 19'], test_size=0.33, random_state=1234)

In [ ]:
X_train, X_test, y2_train, y2_test = train_test_split( X, Y['20 - 39'], test_size=0.33, random_state=1234)

In [ ]:
X_train, X_test, y3_train, y3_test = train_test_split( X, Y['40 - 59'], test_size=0.33, random_state=1234)

In [ ]:
X_train, X_test, y4_train, y4_test = train_test_split( X, Y['60 +'], test_size=0.33, random_state=1234)

### AIC values of different age groups

In [ ]:
sfs_y0 = sfs1.fit(X_train, y0_train)

# output : 0.5312693990784911

In [ ]:
sfs_y1 = sfs1.fit(X_train, y1_train)

# output : 0.5982918872901118

In [ ]:
sfs_y2 = sfs1.fit(X_train, y2_train)

# output : 0.508704493699982

In [ ]:
sfs_y3 = sfs1.fit(X_train, y3_train)

# output : 0.5248464096321375

In [ ]:
sfs_y4 = sfs1.fit(X_train, y4_train)

# output : 0.1061902622061179

#### 0. "All Ages"

In [ ]:
lr_y0 = lr.fit(X_train, y0_train)

In [ ]:
lr_y0_hat=lr_y0.predict(X_test)

In [ ]:
lr_y0_r2 = r2_score(y0_test,lr_y0_hat)

#### 1. "0 - 19"

In [ ]:
lr_y1 = lr.fit(X_train, y1_train)

In [ ]:
lr_y1_hat=lr_y1.predict(X_test)

In [ ]:
lr_y1_r2 = r2_score(y1_test,lr_y1_hat)

#### 2. "20 - 39"

In [ ]:
lr_y2 = lr.fit(X_train, y2_train)

In [ ]:
lr_y2_hat=lr_y2.predict(X_test)

In [ ]:
lr_y2_r2 = r2_score(y2_test,lr_y2_hat)

#### 3. "40 - 59"

In [ ]:
lr_y3 = lr.fit(X_train, y3_train)

In [ ]:
lr_y3_hat=lr_y3.predict(X_test)

In [ ]:
lr_y3_r2 = r2_score(y3_test,lr_y3_hat)

#### 4. "60 +"

In [ ]:
lr_y4 = lr.fit(X_train, y4_train)

In [ ]:
lr_y4_hat=lr_y4.predict(X_test)

In [ ]:
lr_y4_r2 = r2_score(y4_test,lr_y4_hat)

### R-squares of diffent age groups

In [ ]:
[lr_y0_r2,lr_y1_r2,lr_y2_r2,lr_y3_r2,lr_y4_r2]

# output = [0.7370507194297026, 0.4539888252063323, 0.6040683330404932, 0.7734383903795486, 0.5982596341458009]

# Morans' I and LISAs

In [ ]:
essex1=essex.to_crs({'init': 'epsg:27700'})

In [ ]:
essex1=essex1.to_crs({'init': 'epsg:4326'})

In [ ]:
# weights

w = weights.KNN.from_dataframe(essex1, k=5)
w

In [ ]:
w.transform='R'

In [ ]:
y0_lags=np.log(essex1['All Ages'])
y1_lags=np.log(essex1['0 - 19'])
y2_lags=np.log(essex1['20 - 39'])
y3_lags=np.log(essex1['40 - 59'])
y4_lags=np.log(essex1['60 +'])

In [ ]:
essex1['lag_y0']=weights.lag_spatial(w,y0_lags)

In [ ]:
essex1['lag_y1']=weights.lag_spatial(w,y1_lags)

In [ ]:
essex1['lag_y2']=weights.lag_spatial(w,y2_lags)

In [ ]:
essex1['lag_y3']=weights.lag_spatial(w,y3_lags)

In [ ]:
essex1['lag_y4']=weights.lag_spatial(w,y4_lags)

In [ ]:
mi_y0 = esda.Moran(essex1['lag_y0'],w)

In [ ]:
mi_y1 = esda.Moran(essex1['lag_y1'],w)

In [ ]:
mi_y2 = esda.Moran(essex1['lag_y2'],w)

In [ ]:
mi_y3 = esda.Moran(essex1['lag_y3'],w)

In [ ]:
mi_y4 = esda.Moran(essex1['lag_y4'],w)

### Moran's I value and its p-value

In [ ]:
[mi_y0.I,mi_y0.p_sim]

# output : [0.6996827431755985, 0.001]

In [ ]:
[mi_y1.I,mi_y1.p_sim]

# output : [0.7031824977989647, 0.001]

In [ ]:
[mi_y2.I,mi_y2.p_sim]

# output : [0.7126313473981126, 0.001]

In [ ]:
[mi_y3.I,mi_y4.p_sim]

# output : [0.6950751235336577, 0.001]

In [ ]:
[mi_y4.I,mi_y4.p_sim]

# output : [0.6972541451237817, 0.001]

### Clustering visualisation with LISAs 

In [ ]:
lisa_y0 = esda.Moran_Local(essex1['lag_y0'],w)

In [ ]:
lisa_y1 = esda.Moran_Local(essex1['lag_y1'],w)

In [ ]:
lisa_y2 = esda.Moran_Local(essex1['lag_y2'],w)

In [ ]:
lisa_y3 = esda.Moran_Local(essex1['lag_y3'],w)

In [ ]:
lisa_y4 = esda.Moran_Local(essex1['lag_y4'],w)

In [ ]:
essex1['significant_y0']=lisa_y0.p_sim<0.05
essex1['quadrant_y0']=lisa_y0.q

In [ ]:
essex1['significant_y1']=lisa_y1.p_sim<0.05
essex1['quadrant_y1']=lisa_y1.q

In [ ]:
essex1['significant_y2']=lisa_y2.p_sim<0.05
essex1['quadrant_y2']=lisa_y2.q

In [ ]:
essex1['significant_y3']=lisa_y3.p_sim<0.05
essex1['quadrant_y3']=lisa_y3.q

In [ ]:
essex1['significant_y4']=lisa_y4.p_sim<0.05
essex1['quadrant_y4']=lisa_y4.q

In [ ]:
lisa_cluster(lisa_y0,essex1, figsize=(15,10));

In [ ]:
plot_local_autocorrelation(lisa_y0, essex1, 'All Ages');

In [ ]:
plot_local_autocorrelation(lisa_y1, essex1, '0 - 19');

In [ ]:
plot_local_autocorrelation(lisa_y2, essex1, '20 - 39');

In [ ]:
plot_local_autocorrelation(lisa_y3, essex1, '40 - 59');

In [ ]:
plot_local_autocorrelation(lisa_y4, essex1, '60 +');

# Supervised ML Classifications

### conveting the Target variable into Categorical variables

In [ ]:
y0_c=essex['All Ages']
y0_c.describe()

In [ ]:
y1_c=essex['0 - 19']
y1_c.describe()

In [ ]:
y2_c=essex['20 - 39']
y2_c.describe()

In [ ]:
y3_c=essex['40 - 59']
y3_c.describe()

In [ ]:
y4_c=essex['60 +']
y4_c.describe()

In [ ]:
y0_c=pd.cut(y0_c,bins=[0,309.971742, 1474.057691,3046.687207,8005],labels=['low','med','high','v_high'])
y1_c=pd.cut(y1_c,bins=[0,72.316167, 325.499034,740.103625,2465],labels=['low','med','high','v_high'])
y2_c=pd.cut(y2_c,bins=[0,65.190875, 309.003778,792.340802,3840],labels=['low','med','high','v_high'])
y3_c=pd.cut(y3_c,bins=[0,89.610282, 380.038388,789.010829,1832],labels=['low','med','high','v_high'])
y4_c=pd.cut(y4_c,bins=[0,94.833619, 352.472089,760.905416,1928],labels=['low','med','high','v_high'])

### splitting the data into Train and Test data

In [ ]:
X_train, X_test, y0_c_train, y0_c_test = train_test_split( X, y0_c, test_size=0.33, random_state=1234)

In [ ]:
X_train, X_test, y1_c_train, y1_c_test = train_test_split( X, y1_c, test_size=0.33, random_state=1234)

In [ ]:
X_train, X_test, y2_c_train, y2_c_test = train_test_split( X, y2_c, test_size=0.33, random_state=1234)

In [ ]:
X_train, X_test, y3_c_train, y3_c_test = train_test_split( X, y3_c, test_size=0.33, random_state=1234)

In [ ]:
X_train, X_test, y4_c_train, y4_c_test = train_test_split( X, y4_c, test_size=0.33, random_state=1234)

### Traning the data with algorithms

In [ ]:
reg_y0  = reg.fit(X_train,y0_c_train)
rf_y0    = rf.fit(X_train,y0_c_train)
dt_y0    = dt.fit(X_train,y0_c_train)
svm_y0 = svmc.fit(X_train,y0_c_train)

In [ ]:
reg_y1  = reg.fit(X_train,y1_c_train)
rf_y1    = rf.fit(X_train,y1_c_train)
dt_y1    = dt.fit(X_train,y1_c_train)
svm_y1 = svmc.fit(X_train,y1_c_train)

In [ ]:
reg_y2  = reg.fit(X_train,y2_c_train)
rf_y2    = rf.fit(X_train,y2_c_train)
dt_y2    = dt.fit(X_train,y2_c_train)
svm_y2 = svmc.fit(X_train,y2_c_train)

In [ ]:
reg_y3  = reg.fit(X_train,y3_c_train)
rf_y3    = rf.fit(X_train,y3_c_train)
dt_y3    = dt.fit(X_train,y3_c_train)
svm_y3 = svmc.fit(X_train,y3_c_train)

In [ ]:
reg_y4  = reg.fit(X_train,y0_c_train)
rf_y4    = rf.fit(X_train,y0_c_train)
dt_y4    = dt.fit(X_train,y0_c_train)
svm_y4 = svmc.fit(X_train,y0_c_train)

### Predicting the output with trained models

In [ ]:
y0_hat1 = reg_y0.predict(X_test)
y0_hat2 =  rf_y0.predict(X_test)
y0_hat3 =  dt_y0.predict(X_test)
y0_hat4 = svm_y0.predict(X_test)

In [ ]:
y1_hat1 = reg_y1.predict(X_test)
y1_hat2 =  rf_y1.predict(X_test)
y1_hat3 =  dt_y1.predict(X_test)
y1_hat4 = svm_y1.predict(X_test)

In [ ]:
y2_hat1 = reg_y2.predict(X_test)
y2_hat2 =  rf_y2.predict(X_test)
y2_hat3 =  dt_y2.predict(X_test)
y2_hat4 = svm_y2.predict(X_test)

In [ ]:
y3_hat1 = reg_y3.predict(X_test)
y3_hat2 =  rf_y3.predict(X_test)
y3_hat3 =  dt_y3.predict(X_test)
y3_hat4 = svm_y3.predict(X_test)

In [ ]:
y4_hat1 = reg_y4.predict(X_test)
y4_hat2 =  rf_y4.predict(X_test)
y4_hat3 =  dt_y4.predict(X_test)
y4_hat4 = svm_y4.predict(X_test)

### Calculating the Accuracy of each model

In [ ]:
y0_acc1=accuracy_score(y0_c_test,y0_hat1)
y0_acc2=accuracy_score(y0_c_test,y0_hat2)
y0_acc3=accuracy_score(y0_c_test,y0_hat3)
y0_acc4=accuracy_score(y0_c_test,y0_hat4)
y0_acc = [y0_acc1,y0_acc2,y0_acc3,y0_acc4]


In [ ]:
y1_acc1=accuracy_score(y1_c_test,y1_hat1)
y1_acc2=accuracy_score(y1_c_test,y1_hat2)
y1_acc3=accuracy_score(y1_c_test,y1_hat3)
y1_acc4=accuracy_score(y1_c_test,y1_hat4)
y1_acc = [y1_acc1,y1_acc2,y1_acc3,y1_acc4]


In [ ]:
y2_acc1=accuracy_score(y2_c_test,y2_hat1)
y2_acc2=accuracy_score(y2_c_test,y2_hat2)
y2_acc3=accuracy_score(y2_c_test,y2_hat3)
y2_acc4=accuracy_score(y2_c_test,y2_hat4)
y2_acc = [y2_acc1,y2_acc2,y2_acc3,y2_acc4]


In [ ]:
y3_acc1=accuracy_score(y3_c_test,y3_hat1)
y3_acc2=accuracy_score(y3_c_test,y3_hat2)
y3_acc3=accuracy_score(y3_c_test,y3_hat3)
y3_acc4=accuracy_score(y3_c_test,y3_hat4)
y3_acc = [y3_acc1,y3_acc2,y3_acc3,y3_acc4]


In [ ]:
y4_acc1=accuracy_score(y4_c_test,y4_hat1)
y4_acc2=accuracy_score(y4_c_test,y4_hat2)
y4_acc3=accuracy_score(y4_c_test,y4_hat3)
y4_acc4=accuracy_score(y4_c_test,y4_hat4)
y4_acc = [y4_acc1,y4_acc2,y4_acc3,y4_acc4]


In [ ]:
y0_acc

# output = [0.7241, 0.8793, 0.7586, 0.5689]

In [ ]:
y1_acc

# output = [0.7068, 0.8965, 0.8103, 0.6034]

In [ ]:
y2_acc

# output = [0.6896, 0.8448, 0.7931, 0.5689]

In [ ]:
y3_acc

# output = [0.7413, 0.8965, 0.8103, 0.5689]

In [ ]:
y4_acc

# output = [0.6896, 0.7758, 0.6896, 0.5344]